In [ ]:
from matplotlib import pyplot as plt
from rasterio.mask import mask
import geopandas as gpd
import rasterio as rio
import numpy as np
import mpld3
import os

In [ ]:
%matplotlib inline
mpld3.enable_notebook()
plt.rcParams["figure.figsize"] = (9,9)

In [ ]:
name = "Lacey"

# Navigate to the study site data directory
# This directory should contain NAIP, CHM, and boundary files
os.chdir(f"../data/{name}")
current_dir = os.getcwd()
print(f"Working directory: {current_dir}")

In [ ]:
naip_path = r"NAIP_clipped_resampled.tif"
chm_path = r"chm_clipped_resampled.tif"
meadow_extent_path = r"meadow_extent.geojson"

In [ ]:
#with rio.open(naip_path) as src:
#    naip_data = src.read()
#    naip_data = np.moveaxis(naip_data, 0, -1)  # Change from (bands, rows, cols) to (rows, cols, bands)
    
with rio.open(chm_path) as src:
    chm_data = src.read()
    chm_profile = src.profile

chm_data = np.squeeze(chm_data)  # Remove single-dimensional entries
#chm_data = np.where(chm_data < 0, 0, chm_data)  # Set negative values to zero

In [ ]:
# Read NAIP raster and meadow extent
with rio.open(naip_path) as naip_src:
    naip_crs = naip_src.crs

meadow_gdf = gpd.read_file(meadow_extent_path)

# Reproject meadow extent to NAIP CRS if needed
if meadow_gdf.crs != naip_crs:
    meadow_gdf = meadow_gdf.to_crs(naip_crs)

meadow_geom = meadow_gdf.geometry.values

# Clip NAIP image to meadow extent
with rio.open(naip_path) as src:
    naip_data, naip_clipped_transform = mask(src, meadow_geom, crop=True, nodata=0)
    naip_data = np.transpose(naip_data, (1, 2, 0))  # (rows, cols, bands)

In [ ]:
naip_data_float = naip_data.astype(np.float32)
naip_data_float[naip_data_float == 0] = np.nan  # Set zero values to NaN for calculations

In [ ]:
naip_data = np.where(naip_data == 0, np.nan, naip_data)  # Set zero values to NaN

In [ ]:
naip_data.shape, chm_data.shape

NAIP band order = R,G,B,NIR

In [ ]:
R = naip_data[:,:,0]
G = naip_data[:,:,1]
B = naip_data[:,:,2]
NIR = naip_data[:,:,3]

In [ ]:
RGB = np.dstack((R, G, B)).astype(np.uint8)

In [ ]:
# uint8 is the standard 8-bit per channel integer format (0-255) used by most display libraries.
# matplotlib.imshow treats:
#  - uint8 arrays as 0..255 color values per channel,
#  - float arrays as 0.0..1.0 color values per channel (values outside are clipped).
# If your image is stored as floats but not normalized to 0..1, casting to uint8 (or normalizing then casting)
# ensures colors render as intended and avoids silent clipping/contrast issues.
#
# Example: normalize the first three NAIP bands to 0..255 and display as uint8 RGB.

rgb_float = naip_data_float[..., :3]  # (rows, cols, 3) float image with NaNs
# Replace NaNs with 0 (black) for display; choose different fill if desired
rgb_float = np.where(np.isnan(rgb_float), 0.0, rgb_float)

# Heuristic: detect whether floats are already in 0..1 or in 0..255 range and convert accordingly
p99 = np.nanpercentile(rgb_float, 99)
if p99 <= 1.0:
    # floats in 0..1 -> scale to 0..255
    rgb_uint8 = (np.clip(rgb_float, 0.0, 1.0) * 255).astype(np.uint8)
else:
    # floats likely already in 0..255 or other DN range -> rescale by percentile stretch to preserve contrast
    p1 = np.nanpercentile(rgb_float, 1)
    p99 = max(p99, p1 + 1e-6)
    rgb_uint8 = (np.clip((rgb_float - p1) / (p99 - p1), 0.0, 1.0) * 255).astype(np.uint8)

plt.imshow(rgb_uint8)
plt.title("RGB rendered as uint8")
plt.axis("off")
plt.show()

In [ ]:
# using NIR, R, and B bands
NDVI = (NIR - R) / (NIR + R)
NDWI = (R - NIR) / (R + NIR)
brightness = (R + G + B)
avg = (NIR + R + B) / 3
bri_avg = (R - avg) + (G - avg) + (B - avg) / avg
bri_avg2 = ((R - avg) + (G - avg) + (B - avg)) / avg

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(11, 11), sharex=True, sharey=True)
ax[0, 0].imshow(NDVI, cmap='RdYlGn')
ax[0, 0].set_title('NDVI')
ax[0, 1].imshow(NDWI, cmap='RdYlGn')
ax[0, 1].set_title('NDWI')
ax[0, 2].imshow(brightness, cmap='gray')
ax[0, 2].set_title('Brightness')
ax[1, 0].imshow(avg, cmap='gray')
ax[1, 0].set_title('Average Reflectance')
ax[1, 1].imshow(bri_avg, cmap='gray')
ax[1, 1].set_title('Brightness - Average / Average')
ax[1, 2].imshow(bri_avg2, cmap='gray')
ax[1, 2].set_title('Brightness - Average divided by Average')
plt.tight_layout()
plt.show();

In [ ]:
#brightness = naip_data_float[:, :, 0] + naip_data_float[:, :, 1] + naip_data_float[:, :, 2]
#ndvi = (naip_data_float[:, :, 0] - naip_data_float[:, :, 1]) / (naip_data_float[:, :, 0] + naip_data_float[:, :, 1])
#ndwi = (naip_data_float[:, :, 1] - naip_data_float[:, :, 0]) / (naip_data_float[:, :, 1] + naip_data_float[:, :, 0])
#avg = (naip_data_float[:, :, 0] + naip_data_float[:, :, 1] + naip_data_float[:, :, 2]) / 3
#bri_avg = (naip_data_float[:, :, 0] - avg) + (naip_data_float[:, :, 1] - avg) + (naip_data_float[:, :, 2] - avg) / avg

In [ ]:
water = (bri_avg <0) #& (NDWI > 0)

In [ ]:
#bare_mask = (brightness > 320) &(chm_data < 0.5) #& (bri_avg > 29)
bare_mask = (brightness > 320) & (chm_data < 0.5) & ~water & (ndvi < 0.05)  # Adjusted condition to include NDVI threshold
bare_classified = np.zeros(chm_data.shape, dtype=np.uint8)
bare_classified[bare_mask] = 1  # Class 1 for bare
bare_classified[~bare_mask] = 2  # Class 2 for non-bare
bare_classified = np.expand_dims(bare_classified, axis=0)  # Add channel dimension

In [ ]:
with rio.open("bare_classified.tif", "w", **chm_profile) as dst:
    dst.write(bare_classified)

## buffer main channel to exclude bare earth within the main channel

In [ ]:

import geopandas as gpd
import rasterio
from rasterio.mask import mask
from shapely.geometry import mapping
import numpy as np

def mask_inside_buffer(raster_path, polyline_path, output_path, buffer_distance=10):
    # Step 1: Read the polyline and buffer it
    gdf = gpd.read_file(polyline_path)
    gdf_buffered = gdf.copy()
    gdf_buffered['geometry'] = gdf.geometry.buffer(buffer_distance)

    # Step 2: Open the raster
    with rasterio.open(raster_path) as src:
        raster_data = src.read(1)
        raster_meta = src.meta.copy()
        raster_crs = src.crs

        # Step 3: Reproject buffered geometry to match raster CRS
        if gdf_buffered.crs != raster_crs:
            gdf_buffered = gdf_buffered.to_crs(raster_crs)

        # Step 4: Mask the raster INSIDE the buffer (invert=False means mask inside)
        out_image, out_transform = mask(
            src,
            [mapping(geom) for geom in gdf_buffered.geometry],
            invert=True,  # Keep data OUTSIDE the buffer
            crop=False,
            filled=True,
            nodata=src.nodata
        )

    # Step 5: Update metadata and write output
    raster_meta.update({
        "height": out_image.shape[1],
        "width": out_image.shape[2],
        "transform": src.transform,
        "dtype": rasterio.float32
    })

    with rasterio.open(output_path, "w", **raster_meta) as dst:
        dst.write(out_image.astype(rasterio.float32))

    return f"Raster saved to {output_path}, with buffer area masked out."


In [ ]:
mask_inside_buffer('bare_classified.tif', r"..\lacey_plong_simp.geojson", 'bare_classified_clipped.tif')

## Sumary -%cover:  
trees, shrubs, bare earth

In [ ]:
with rio.open('bare_classified_clipped.tif') as src:
    bare_classified_clipped = src.read()
    pixel = src.transform.a
    bare_earth_area = np.sum(bare_classified_clipped == 1) * pixel**2

In [ ]:
with rio.open("CHM_clipped.tif") as src:
    chm_data = src.read()
    pixel = src.transform.a
    trees = np.sum(chm_data > 5) * pixel**2
    shrubs = np.sum(((chm_data > 0.2) & (chm_data < 5))) * pixel**2
    total_area = np.sum(~np.isnan(chm_data)) * pixel**2

In [ ]:
bare_earth_area

In [ ]:
round(bare_earth_area / total_area * 100, 2)

In [ ]:
round(trees / total_area * 100, 2)

In [ ]:
round(shrubs / total_area * 100, 2)